In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import datetime
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
import sqlite3 as sq
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

Quarterly Employment During the Heart of COVID 19 Pandemic

The Woods & Poole data that is used more at the county level is only used at the place level for total employment projections. This is for consistency. This is a land use model ultimately, so JobsEQ employment is selected to model place level employment for place of work. This data is derived from the QCEW and other inputs so is selected as a better option - both are modeled so we use the one modeled directly from UI claims rather than land use at this granular of a geography.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  

*load the following query: JEQQuarterly_Industry*

+ go to "Draft Mode"
+ Select the "Industry" dataset, quarterly  
+ Select the following dimensions: Region, Industry, Year: fill regions with all geographies including US, State, GNRC (13), MPO (7), GNRC Region (14) and sub all counties, all places  
+ Fill the year/quarter with 2019-current (in the future change this to relevant time period if need be)  
+ select total and high level industries as listed below:  

Industries to Select:  
+ Total - All Industries  
(Custom Created Groups *begin to type them as they appear here and they will appear in the search bar, if not then remake the custom group as described below*)  
+ Snapshot 11: Construction (23: Construction)  
+ Snapshot 11: Manufacturing (31: Manufacturing)  
+ Snapshot 11: Information (51: Information)  
+ Snapshot 11: Public Administration (92: Public Administration)  
+ Snapshot 11: Other/Unclassified (81: Other, Except Public Administration, 99: Unclassified)  
+ Snapshot 11: Leisure & Hospitality (71: Arts, Entertainment, & Recreation, 72: Accommodation & Food Services)   
+ Snapshot 11: Education & Health Services (61: Education, 62: Healthcare & Social Assistance)   
+ Snapshot 11: Professional & Business Services (54: Professional, Scientific, & Technical, 55: Management of Companies & Enterprises, 56: Administrative & Support & Waste Management & Remediation)   
+ Snapshot 11: Financial Activities (52: Finance & Insurance, 53: Real Estate & Rental & Leasing)   
+ Snapshot 11: Trade, Transportation & Utilities (42: Wholesale Trade, 44-45: Retail Trade, 48-49: Transportation & Warehousing, 22: Utilities)  
+ Snapshot 11: Natural Resources & Mining (11: Agricultural Activities, Forestry, Fishing, & Hunting, 21: Mining, Quarrying, Oil & Gas) 
+ Snapshot: White Collar  
+ Snapshot: Blue Collar  
+ Snapshot 4: Government  
+ Snapshot 4: Retail & Hospitality  
+ Snapshot 4: Professional Services  
+ Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics  

+ Select the following values: "Employment (4Q Moving Average)"  

Make sure region and year/quarter are on the side, and industry data is on the top.  

*Resume here after loading query and downloading data.*

Once it is completed, open and delete the top row that says "Industry Data", and adjust so that the column headers reflect the industry, as it is all the same datapoint. Delete the FIPS column. Rename the Region column to NAME and the quarter column to Time, delete the data notes at the bottom. Save as:   "JobsEQ_IndustryEmploymentCOVIDQuarterly_AllGeos.csv"

In [83]:
#import data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_IndustryEmploymentCOVIDQuarterly_AllGeos.csv')
data.head(2)

,NAME,Time,Total - All Industries,Snapshot 11: Construction,Snapshot 11: Manufacturing,Snapshot 11: Information,Snapshot 11: Public Administration,Snapshot 11: Other/Unclassified,Snapshot 11: Leisure & Hospitality,Snapshot 11: Education & Health Services,Snapshot 11: Professional & Business Services,Snapshot 11: Financial Activities,"Snapshot 11: Trade, Transportation, & Utilities",Snapshot 11: Natural Resources & Mining,Snapshot: White Collar,Snapshot: Blue Collar,Snapshot 4: Government,Snapshot 4: Retail & Hospitality,Snapshot 4: Professional Services,"Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics"
0,USA,2019Q1,158025474.3,9094815.083,13015409.73,3081771.173,7426728.447,7243072.853,17441948.18,35570287.26,23193833.42,8966303.726,30156661.55,2834642.848,68078678.69,36491771.95,7426728.447,30687697.70,81276170.22,38634877.91
1,USA,2019Q2,158445187.9,9153194.347,13056874.75,3089323.039,7440929.506,7233337.469,17510564.87,35698465.84,23276911.82,8989722.209,30168440.87,2827423.193,68320200.13,36669818.13,7440929.506,30679588.15,81522751.89,38801918.36


In [84]:
#quickly make sure the geographies are the full census geographies
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus)
data = data.transpose().reset_index(drop = False)

In [85]:
#make a list of the times to loop through unique instances for aggregating geographies later
times = list(data['Time'].unique())

In [86]:
#set the index to name and time, creating unique instances for each industry group, transpose, and make sure the columns are float datatypes
data = data.set_index(['NAME', 'Time']).transpose()
cols = data.columns
data[cols] = data[cols].astype(float)

In [87]:
data.head(2)

NAME                      United States                                                                                                    \
Time                             2019Q1        2019Q2        2019Q3        2019Q4        2020Q1        2020Q2        2020Q3        2020Q4   
Total - All Industries     1.580255e+08  1.584452e+08  1.589213e+08  1.593484e+08  1.596866e+08  1.551281e+08  1.522853e+08  1.500329e+08   
Snapshot 11: Construction  9.094815e+06  9.153194e+06  9.218998e+06  9.269805e+06  9.313434e+06  9.135578e+06  9.037456e+06  8.965809e+06   

NAME                                                                                                                                       \
Time                             2021Q1        2021Q2        2021Q3        2021Q4        2022Q1        2022Q2        2022Q3        2022Q4   
Total - All Industries     1.480742e+08  1.514055e+08  1.534785e+08  1.553502e+08  1.572706e+08  1.590211e+08  1.604610e+08  1.614844e+08   
Snapshot 11: Construction  8.915466e+06  9.107387e+06  9.218495e+06  9.314620e+06  9.400457e+06  9.477380e+06  9.536480e+06  9.584749e+06   

NAME                                        Tennessee                                                                                      \
Time                             2023Q1        2019Q1        2019Q2        2019Q3        2019Q4        2020Q1        2020Q2        2020Q3   
Total - All Industries     1.624729e+08  3.262321e+06  3.275649e+06  3.290374e+06  3.301519e+06  3.311207e+06  3.238563e+06  3.199233e+06   
Snapshot 11: Construction  9.648245e+06  1.716014e+05  1.732020e+05  1.745590e+05  1.759428e+05  1.769771e+05  1.754061e+05  1.746974e+05   

NAME                                                                                                                                      \
Time                             2020Q4        2021Q1        2021Q2       2021Q3        2021Q4        2022Q1        2022Q2        2022Q3   
Total - All Industries     3.177211e+06  3.156717e+06  3.220522e+06  3262395.078  3.290514e+06  3.327386e+06  3.367457e+06  3.400365e+06   
Snapshot 11: Construction  1.744549e+05  1.749226e+05  1.781297e+05   181325.948  1.841253e+05  1.868308e+05  1.896095e+05  1.917777e+05   

NAME                                                           GNRC                                                                        \
Time                             2022Q4        2023Q1        2019Q1        2019Q2        2019Q3        2019Q4        2020Q1        2020Q2   
Total - All Industries     3.430284e+06  3.454684e+06  1.068697e+06  1.077315e+06  1.085972e+06  1.093982e+06  1.099465e+06  1.071861e+06   
Snapshot 11: Construction  1.939177e+05  1.960737e+05  5.833100e+04  5.915915e+04  5.991592e+04  6.080827e+04  6.133967e+04  6.093216e+04   

NAME                                                                                                                                       \
Time                             2020Q3        2020Q4        2021Q1        2021Q2        2021Q3        2021Q4        2022Q1        2022Q2   
Total - All Industries     1.055979e+06  1.046672e+06  1.038467e+06  1.064105e+06  1.082446e+06  1.095739e+06  1.111680e+06  1.127780e+06   
Snapshot 11: Construction  6.073438e+04  6.066902e+04  6.093620e+04  6.206173e+04  6.334314e+04  6.462324e+04  6.587773e+04  6.713576e+04   

NAME                                                                          MPO                                                          \
Time                             2022Q3        2022Q4        2023Q1        2019Q1        2019Q2        2019Q3        2019Q4        2020Q1   
Total - All Industries     1.141572e+06  1.152572e+06  1.161777e+06  1.004288e+06  1.013034e+06  1.021690e+06  1.029150e+06  1.034446e+06   
Snapshot 11: Construction  6.824509e+04  6.922829e+04  7.012701e+04  5.302547e+04  5.382014e+04  5.456217e+04  5.545926e+04  5.599913e+04   

NAME                                             

In [88]:
#loop through the times we listed earlier to aggregate the incorporated and unincorporated areas
for time in times:
    RuthInc = [data[('Eagleville city, Tennessee', '{}'.format(time))],data[('La Vergne city, Tennessee', '{}'.format(time))],
               data[('Murfreesboro city, Tennessee', '{}'.format(time))],data[('Smyrna town, Tennessee', '{}'.format(time))]]
    data[('Rutherford Incorporated', '{}'.format(time))] = sum(RuthInc)
    data[('Rutherford Unincorporated', '{}'.format(time))] = data[('Rutherford County, Tennessee', '{}'.format(time))] - data[('Rutherford Incorporated', '{}'.format(time))]
    WilsonInc = [data[('Lebanon city, Tennessee', '{}'.format(time))],data[('Mount Juliet city, Tennessee', '{}'.format(time))],
                 data[('Watertown city, Tennessee', '{}'.format(time))]]
    data[('Wilson Incorporated', '{}'.format(time))] = sum(WilsonInc)
    data[('Wilson Unincorporated', '{}'.format(time))] = data[('Wilson County, Tennessee', '{}'.format(time))] - data[('Wilson Incorporated', '{}'.format(time))]
    CheathInc = [data[('Ashland City town, Tennessee', '{}'.format(time))],data[('Kingston Springs town, Tennessee', '{}'.format(time))],
                 data[('Pegram town, Tennessee', '{}'.format(time))],data[('Pleasant View city, Tennessee', '{}'.format(time))]]
    data[('Cheatham Incorporated', '{}'.format(time))] = sum(CheathInc)
    data[('Cheatham Unincorporated', '{}'.format(time))] = data[('Cheatham County, Tennessee', '{}'.format(time))] - data[('Cheatham Incorporated', '{}'.format(time))]
    DicksInc = [data[('Burns town, Tennessee', '{}'.format(time))],data[('Charlotte town, Tennessee', '{}'.format(time))],
                data[('Dickson city, Tennessee', '{}'.format(time))],data[('Slayden town, Tennessee', '{}'.format(time))],
                data[('Vanleer town, Tennessee', '{}'.format(time))],data[('White Bluff town, Tennessee', '{}'.format(time))]]
    data[('Dickson Incorporated', '{}'.format(time))] = sum(DicksInc)
    data[('Dickson Unincorporated', '{}'.format(time))] = data[('Dickson County, Tennessee', '{}'.format(time))] - data[('Dickson Incorporated', '{}'.format(time))]
    HumphInc = [data[('McEwen city, Tennessee', '{}'.format(time))],data[('New Johnsonville city, Tennessee', '{}'.format(time))],
                data[('Waverly city, Tennessee', '{}'.format(time))]]
    data[('Humphreys Incorporated', '{}'.format(time))] = sum(HumphInc)
    data[('Humphreys Unincorporated', '{}'.format(time))] = data[('Humphreys County, Tennessee', '{}'.format(time))] - data[('Humphreys Incorporated', '{}'.format(time))]
    data[('Montgomery Incorporated', '{}'.format(time))] = data[('Clarksville city, Tennessee', '{}'.format(time))]
    data[('Montgomery Unincorporated', '{}'.format(time))] = data[('Montgomery County, Tennessee', '{}'.format(time))] - data[('Montgomery Incorporated', '{}'.format(time))]

In [89]:
#transpose and check
data = data.transpose()
data.head(2)

Total - All Industries  Snapshot 11: Construction  Snapshot 11: Manufacturing  Snapshot 11: Information  \
NAME          Time                                                                                                              
United States 2019Q1             158025474.3                9094815.083                 13015409.73               3081771.173   
              2019Q2             158445187.9                9153194.347                 13056874.75               3089323.039   

                      Snapshot 11: Public Administration  Snapshot 11: Other/Unclassified  Snapshot 11: Leisure & Hospitality  \
NAME          Time                                                                                                              
United States 2019Q1                         7426728.447                      7243072.853                         17441948.18   
              2019Q2                         7440929.506                      7233337.469                         17510564.87   

                      Snapshot 11: Education & Health Services  Snapshot 11: Professional & Business Services  Snapshot 11: Financial Activities  \
NAME          Time                                                                                                                                 
United States 2019Q1                               35570287.26                                    23193833.42                        8966303.726   
              2019Q2                               35698465.84                                    23276911.82                        8989722.209   

                      Snapshot 11: Trade, Transportation, & Utilities  Snapshot 11: Natural Resources & Mining  Snapshot: White Collar  \
NAME          Time                                                                                                                       
United States 2019Q1                                      30156661.55                              2834642.848             68078678.69   
              2019Q2                                      30168440.87                              2827423.193             68320200.13   

                      Snapshot: Blue Collar  Snapshot 4: Government  Snapshot 4: Retail & Hospitality  Snapshot 4: Professional Services  \
NAME          Time                                                                                                                         
United States 2019Q1            36491771.95             7426728.447                       30687697.70                        81276170.22   
              2019Q2            36669818.13             7440929.506                       30679588.15                        81522751.89   

                      Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics  
NAME          Time                                                                   
United States 2019Q1                                        38634877.91              
              2019Q2                                        38801918.36

In [90]:
#sum these, note that this categorization for service producing doesn't contain trade, transportation & utilities as that's broken out separately
thelist = [data['Snapshot 11: Natural Resources & Mining'], data['Snapshot 11: Construction'], data['Snapshot 11: Manufacturing']]
data['Goods Producing'] = sum(thelist)
thelist = [data['Snapshot 11: Information'], data['Snapshot 11: Financial Activities'], data['Snapshot 11: Professional & Business Services'], 
           data['Snapshot 11: Education & Health Services'], data['Snapshot 11: Leisure & Hospitality'], data['Snapshot 11: Public Administration'], 
           data['Snapshot 11: Other/Unclassified']]
data['Service Producing'] = sum(thelist)

In [91]:
#resum so that percentages work - data caveat that they would not sum perfectly due to rounding
thelist = [data['Snapshot 11: Construction'], data['Snapshot 11: Manufacturing'], 
           data['Snapshot 11: Information'], data['Snapshot 11: Public Administration'], 
           data['Snapshot 11: Other/Unclassified'], data['Snapshot 11: Leisure & Hospitality'],
           data['Snapshot 11: Education & Health Services'], data['Snapshot 11: Professional & Business Services'], 
           data['Snapshot 11: Financial Activities'], data['Snapshot 11: Trade, Transportation, & Utilities'], data['Snapshot 11: Natural Resources & Mining']]
data['Total - All Industries'] = sum(thelist)

In [92]:
#rename columns
data = data.rename(columns = {'Total - All Industries': 'Total', 'Snapshot 11: Construction': 'Construction',
                              'Snapshot 11: Manufacturing': 'Manufacturing', 'Snapshot 11: Information': 'Information', 
                              'Snapshot 11: Public Administration': 'Public Administration', 
                              'Snapshot 11: Other/Unclassified': 'Other/Unclassified', 
                              'Snapshot 11: Leisure & Hospitality': 'Leisure & Hospitality', 
                              'Snapshot 11: Education & Health Services': 'Education & Health Services', 
                              'Snapshot 11: Professional & Business Services': 'Professional & Business Services', 
                              'Snapshot 11: Financial Activities': 'Financial Activities', 
                              'Snapshot 11: Trade, Transportation, & Utilities': 'Trade, Transportation, & Utilities', 
                              'Snapshot 11: Natural Resources & Mining': 'Natural Resources & Mining'
                             })

In [93]:
#set the index to year and name so that we can add a prefix to all of the other columns, before resetting the index without dropping
data.reset_index(drop = False, inplace = True)
data = data.set_index(['NAME', 'Time']).add_prefix('Ind:').reset_index(drop = False)

In [94]:
#name cols as all of the columns and then go from wide to long format with .melt(), call this thing because it gets buggy if you call if data 
cols = data.columns
thing = data.melt(var_name = 'Industry', id_vars = ['NAME', 'Time'], value_vars = cols, value_name = 'Employment')

In [95]:
thing.head(2)

,NAME,Time,Industry,Employment
0,United States,2019Q1,Ind:Total,1.580255e+08
1,United States,2019Q2,Ind:Total,1.584452e+08


In [96]:
#create a separate dataframe where we will calculate percent of the total for each occupation group
percents = data

In [97]:
#calculate percent of total
percents['Ind:Construction'] = percent(percents['Ind:Construction'], percents['Ind:Total'])
percents['Ind:Manufacturing'] = percent(percents['Ind:Manufacturing'], percents['Ind:Total'])
percents['Ind:Information'] = percent(percents['Ind:Information'], percents['Ind:Total'])
percents['Ind:Public Administration'] = percent(percents['Ind:Public Administration'], percents['Ind:Total'])
percents['Ind:Other/Unclassified'] = percent(percents['Ind:Other/Unclassified'], percents['Ind:Total'])
percents['Ind:Leisure & Hospitality'] = percent(percents['Ind:Leisure & Hospitality'], percents['Ind:Total'])
percents['Ind:Education & Health Services'] = percent(percents['Ind:Education & Health Services'], percents['Ind:Total'])
percents['Ind:Professional & Business Services'] = percent(percents['Ind:Professional & Business Services'], percents['Ind:Total'])
percents['Ind:Financial Activities'] = percent(percents['Ind:Financial Activities'], percents['Ind:Total'])
percents['Ind:Trade, Transportation, & Utilities'] = percent(percents['Ind:Trade, Transportation, & Utilities'], percents['Ind:Total'])
percents['Ind:Natural Resources & Mining'] = percent(percents['Ind:Natural Resources & Mining'], percents['Ind:Total'])
percents['Ind:Goods Producing'] = percent(percents['Ind:Goods Producing'], percents['Ind:Total'])
percents['Ind:Service Producing'] = percent(percents['Ind:Service Producing'], percents['Ind:Total'])
percents['Ind:Snapshot: White Collar'] = percent(percents['Ind:Snapshot: White Collar'], percents['Ind:Total'])
percents['Ind:Snapshot: Blue Collar'] = percent(percents['Ind:Snapshot: Blue Collar'], percents['Ind:Total'])
percents['Ind:Snapshot 4: Government'] = percent(percents['Ind:Snapshot 4: Government'], percents['Ind:Total'])
percents['Ind:Snapshot 4: Retail & Hospitality'] = percent(percents['Ind:Snapshot 4: Retail & Hospitality'], percents['Ind:Total'])
percents['Ind:Snapshot 4: Professional Services'] = percent(percents['Ind:Snapshot 4: Professional Services'], percents['Ind:Total'])
percents['Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics'] = percent(percents['Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics'],
                                                                                        percents['Ind:Total'])

In [98]:
#similarly melt this dataframe
cols = percents.columns
percents = percents.reset_index(drop = False).melt(var_name = 'Industry', id_vars = ['NAME', 'Time'], value_vars = cols, value_name = 'Percent of Employment')
percents.head(2)

,NAME,Time,Industry,Percent of Employment
0,United States,2019Q1,Ind:Total,1.580255e+08
1,United States,2019Q2,Ind:Total,1.584452e+08


In [99]:
#we never renamed the total column when we duplicated the dataframe for percents, we want this to be null so just transpose, drop it, and transpose back
percents = percents.set_index('Industry').transpose()
percents = percents.drop(columns = 'Ind:Total')
percents = percents.transpose().reset_index(drop = False)

In [100]:
#merge total employment and percent of total employment
data = thing.merge(percents, on = ['Industry', 'Time', 'NAME'], how = 'outer')
data.head(3)

,NAME,Time,Industry,Employment,Percent of Employment
0,United States,2019Q1,Ind:Total,1.580255e+08,NaN
1,United States,2019Q2,Ind:Total,1.584452e+08,NaN
2,United States,2019Q3,Ind:Total,1.589213e+08,NaN


In [101]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos beforeresetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose().reset_index(drop = False)

In [102]:
#set the index to "index" (the variable names) and transpose, then reset the index without dropping it
data = data.set_index('index').transpose().reset_index(drop = False)

In [103]:
data.head(2)

index,NAME,Time,Industry,Employment,Percent of Employment
0,United States,2019Q1,Ind:Total,158025474.27,NaN
1,United States,2019Q2,Ind:Total,158445187.913,NaN


In [104]:
data.tail(2)

index,NAME,Time,Industry,Employment,Percent of Employment
29238,Montgomery Incorporated,2023Q1,Ind:Service Producing,37201.449949,66.818321
29239,Montgomery Unincorporated,2023Q1,Ind:Service Producing,4463.849064,40.866939


In [105]:
#go long to wide
data = pd.pivot(data, index = ['NAME', 'Time'], columns = 'Industry', values = ['Employment', 'Percent of Employment'])

In [106]:
#we'll have to flatten the multicolumns and then rename them to match the header formatting of our other data
data.columns = data.columns.to_flat_index()

In [107]:
#rename
inddict = {('Employment', 'Ind:Education & Health Services'): 'Ind:Education & Health Services', 
           ('Employment', 'Ind:Construction'): 'Ind:Construction', 
           ('Employment', 'Ind:Financial Activities'): 'Ind:Financial Activities',
           ('Employment', 'Ind:Information'): 'Ind:Information', 
           ('Employment', 'Ind:Manufacturing'): 'Ind:Manufacturing', 
           ('Employment', 'Ind:Other/Unclassified'): 'Ind:Other/Unclassified', 
           ('Employment', 'Ind:Public Administration'): 'Ind:Public Administration', 
           ('Employment', 'Ind:Total'): 'Ind:Total', 
           ('Employment', 'Ind:Goods Producing'): 'Ind:Goods Producing', 
           ('Employment', 'Ind:Leisure & Hospitality'): 'Ind:Leisure & Hospitality', 
           ('Employment', 'Ind:Natural Resources & Mining'): 'Ind:Natural Resources & Mining', 
           ('Employment', 'Ind:Professional & Business Services'): 'Ind:Professional & Business Services', 
           ('Employment', 'Ind:Service Producing'): 'Ind:Service Producing', 
           ('Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind:Trade, Transportation, & Utilities',
           ('Employment', 'Ind:Snapshot: White Collar'): 'Ind:Snapshot: White Collar', 
           ('Employment', 'Ind:Snapshot: Blue Collar'): 'Ind:Snapshot: Blue Collar', 
           ('Employment', 'Ind:Snapshot 4: Government'): 'Ind:Snapshot 4: Government', 
           ('Employment', 'Ind:Snapshot 4: Retail & Hospitality'): 'Ind:Snapshot 4: Retail & Hospitality', 
           ('Employment', 'Ind:Snapshot 4: Professional Services'): 'Ind:Snapshot 4: Professional Services', 
           ('Employment', 'Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics',
           ('Percent of Employment', 'Ind:Education & Health Services'): 'Ind%:Education & Health Services',
           ('Percent of Employment', 'Ind:Construction'): 'Ind%:Construction', 
           ('Percent of Employment', 'Ind:Financial Activities'): 'Ind%:Financial Activities',
           ('Percent of Employment', 'Ind:Information'): 'Ind%:Information', 
           ('Percent of Employment', 'Ind:Manufacturing'): 'Ind%:Manufacturing', 
           ('Percent of Employment', 'Ind:Other/Unclassified'): 'Ind%:Other/Unclassified', 
           ('Percent of Employment', 'Ind:Public Administration'): 'Ind%:Public Administration', 
           ('Percent of Employment', 'Ind:Total'): 'Ind%:Total', 
           ('Percent of Employment', 'Ind:Goods Producing'): 'Ind%:Goods Producing', 
           ('Percent of Employment', 'Ind:Leisure & Hospitality'): 'Ind%:Leisure & Hospitality', 
           ('Percent of Employment', 'Ind:Natural Resources & Mining'): 'Ind%:Natural Resources & Mining', 
           ('Percent of Employment', 'Ind:Professional & Business Services'): 'Ind%:Professional & Business Services', 
           ('Percent of Employment', 'Ind:Service Producing'): 'Ind%:Service Producing', 
           ('Percent of Employment', 'Ind:Trade, Transportation, & Utilities'): 'Ind%:Trade, Transportation, & Utilities', 
           ('Percent of Employment', 'Ind:Snapshot: White Collar'): 'Ind%:Snapshot: White Collar', 
           ('Percent of Employment', 'Ind:Snapshot: Blue Collar'): 'Ind%:Snapshot: Blue Collar', 
           ('Percent of Employment', 'Ind:Snapshot 4: Government'): 'Ind%:Snapshot 4: Government', 
           ('Percent of Employment', 'Ind:Snapshot 4: Retail & Hospitality'): 'Ind%:Snapshot 4: Retail & Hospitality', 
           ('Percent of Employment', 'Ind:Snapshot 4: Professional Services'): 'Ind%:Snapshot 4: Professional Services', 
           ('Percent of Employment', 'Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics'): 'Ind%:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics',
           }
data = data.rename(columns = inddict)
data = data.reset_index(drop = False)

In [108]:
data.head(2)

,NAME,Time,Ind:Construction,Ind:Education & Health Services,Ind:Financial Activities,Ind:Goods Producing,Ind:Information,Ind:Leisure & Hospitality,Ind:Manufacturing,Ind:Natural Resources & Mining,Ind:Other/Unclassified,Ind:Professional & Business Services,Ind:Public Administration,Ind:Service Producing,Ind:Snapshot 4: Government,"Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics",Ind:Snapshot 4: Professional Services,Ind:Snapshot 4: Retail & Hospitality,Ind:Snapshot: Blue Collar,Ind:Snapshot: White Collar,Ind:Total,"Ind:Trade, Transportation, & Utilities",Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Snapshot 4: Government,"Ind%:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics",Ind%:Snapshot 4: Professional Services,Ind%:Snapshot 4: Retail & Hospitality,Ind%:Snapshot: Blue Collar,Ind%:Snapshot: White Collar,Ind%:Total,"Ind%:Trade, Transportation, & Utilities"
0,"Adams city, Tennessee",2019Q1,1.268832,14.583774,3.091802,2.030991,0.005335,8.734091,0.000528,0.761631,2.039257,0.861694,3.644034,32.959987,3.644034,3.579932,20.886733,29.985928,2.818301,21.905574,58.096627,23.10565,2.184002,25.102617,5.321827,3.495884,0.009183,15.033731,0.00091,1.310972,3.510113,1.483208,6.272368,56.733047,6.272368,6.16203,35.951713,51.613888,4.851058,37.705414,NaN,39.771069
1,"Adams city, Tennessee",2019Q2,0.932532,13.736945,3.136402,1.682612,0.010276,8.2936,0.000509,0.749571,2.067902,0.899265,3.321756,31.466146,3.321756,3.110981,20.16536,29.421939,2.36141,20.78431,56.020036,22.871277,1.664641,24.521485,5.598714,3.00359,0.018343,14.804704,0.000909,1.33804,3.691362,1.605257,5.929586,56.16945,5.929586,5.553336,35.996692,52.520387,4.215295,37.101565,NaN,40.82696


Use datetime to convert the Time column to a quarterly period.

https://stackoverflow.com/questions/53898482/clean-way-to-convert-quarterly-periods-to-datetime-in-pandas

In [109]:
qs = data['Time'].str.replace(r'(Q\d) (\d+)', r'\2-\1')

C:\Users\jmccall\AppData\Local\Temp\ipykernel_10672\2473378646.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  qs = data['Time'].str.replace(r'(Q\d) (\d+)', r'\2-\1')


In [110]:
#create a time stamp
data['DateTime'] = pd.PeriodIndex(qs, freq = 'Q').to_timestamp()

In [111]:
#before adding other identifier columns convert numerical to float
cols = list(data.columns)
cols.remove('NAME')
cols.remove('Time')
cols.remove('DateTime')
data[cols] = data[cols].astype(float)

In [112]:
#check that this worked
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 43 columns):
 #   Column                                                              Non-Null Count  Dtype         
---  ------                                                              --------------  -----         
 0   NAME                                                                1462 non-null   object        
 1   Time                                                                1462 non-null   object        
 2   Ind:Construction                                                    1462 non-null   float64       
 3   Ind:Education & Health Services                                     1462 non-null   float64       
 4   Ind:Financial Activities                                            1462 non-null   float64       
 5   Ind:Goods Producing                                                 1462 non-null   float64       
 6   Ind:Information                                         

In [113]:
#insert the datetime after the geography name and drop the datetime column
data.insert(1, 'TimeStamp', data['DateTime'])
data = data.drop(columns = 'DateTime')

In [114]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [115]:
data.head()

,NAME,TimeStamp,Time,Ind:Construction,Ind:Education & Health Services,Ind:Financial Activities,Ind:Goods Producing,Ind:Information,Ind:Leisure & Hospitality,Ind:Manufacturing,Ind:Natural Resources & Mining,Ind:Other/Unclassified,Ind:Professional & Business Services,Ind:Public Administration,Ind:Service Producing,Ind:Snapshot 4: Government,"Ind:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics",Ind:Snapshot 4: Professional Services,Ind:Snapshot 4: Retail & Hospitality,Ind:Snapshot: Blue Collar,Ind:Snapshot: White Collar,Ind:Total,"Ind:Trade, Transportation, & Utilities",Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Snapshot 4: Government,"Ind%:Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics",Ind%:Snapshot 4: Professional Services,Ind%:Snapshot 4: Retail & Hospitality,Ind%:Snapshot: Blue Collar,Ind%:Snapshot: White Collar,Ind%:Total,"Ind%:Trade, Transportation, & Utilities",GEO_ID,Source
0,"Adams city, Tennessee",2019-01-01,2019Q1,1.268832,14.583774,3.091802,2.030991,0.005335,8.734091,0.000528,0.761631,2.039257,0.861694,3.644034,32.959987,3.644034,3.579932,20.886733,29.985928,2.818301,21.905574,58.096627,23.105650,2.184002,25.102617,5.321827,3.495884,0.009183,15.033731,0.000910,1.310972,3.510113,1.483208,6.272368,56.733047,6.272368,6.162030,35.951713,51.613888,4.851058,37.705414,NaN,39.771069,1600000US4700200,JobsEQ
1,"Adams city, Tennessee",2019-04-01,2019Q2,0.932532,13.736945,3.136402,1.682612,0.010276,8.293600,0.000509,0.749571,2.067902,0.899265,3.321756,31.466146,3.321756,3.110981,20.165360,29.421939,2.361410,20.784310,56.020036,22.871277,1.664641,24.521485,5.598714,3.003590,0.018343,14.804704,0.000909,1.338040,3.691362,1.605257,5.929586,56.169450,5.929586,5.553336,35.996692,52.520387,4.215295,37.101565,NaN,40.826960,1600000US4700200,JobsEQ
2,"Adams city, Tennessee",2019-07-01,2019Q3,0.599508,12.901809,3.163929,1.348407,0.018164,7.948416,0.000493,0.748406,2.131102,0.946073,2.997581,30.107073,2.997581,2.641716,19.482272,28.718428,1.893310,19.663294,53.839996,22.384516,1.113499,23.963242,5.876541,2.504471,0.033737,14.763032,0.000916,1.390056,3.958213,1.757193,5.567572,55.919530,5.567572,4.906605,36.185500,53.340324,3.516549,36.521723,NaN,41.575999,1600000US4700200,JobsEQ
3,"Adams city, Tennessee",2019-10-01,2019Q4,0.265483,12.183179,3.155745,1.004813,0.025685,7.585570,0.000476,0.738853,2.097124,0.990559,2.672522,28.710385,2.672522,2.151460,18.781177,27.709064,1.412607,18.619898,51.314224,21.599027,0.517368,23.742303,6.149845,1.958157,0.050054,14.782588,0.000928,1.439861,4.086828,1.930380,5.208151,55.950149,5.208151,4.192717,36.600334,53.998798,2.752856,36.286036,NaN,42.091695,1600000US4700200,JobsEQ
4,"Adams city, Tennessee",2020-01-01,2020Q1,0.575134,9.787877,3.317701,1.288758,0.027968,7.542050,0.000475,0.713148,2.384248,0.930012,2.481662,26.471519,2.481662,2.358832,16.788321,27.749891,1.645683,16.198797,49.378706,21.618429,1.164742,19.822061,6.718891,2.609947,0.056640,15.273892,0.000963,1.444243,4.828493,1.883428,5.025775,53.609179,5.025775,4.777022,33.999111,56.198092,3.332780,32.805229,NaN,43.780874,1600000US4700200,JobsEQ


In [43]:
#final check
data.head()

,NAME,TimeStamp,Time,Ind:Construction,Ind:Education & Health Services,Ind:Financial Activities,Ind:Goods Producing,Ind:Information,Ind:Leisure & Hospitality,Ind:Manufacturing,Ind:Natural Resources & Mining,Ind:Other/Unclassified,Ind:Professional & Business Services,Ind:Public Administration,Ind:Service Producing,Ind:Total,"Ind:Trade, Transportation, & Utilities",Ind%:Construction,Ind%:Education & Health Services,Ind%:Financial Activities,Ind%:Goods Producing,Ind%:Information,Ind%:Leisure & Hospitality,Ind%:Manufacturing,Ind%:Natural Resources & Mining,Ind%:Other/Unclassified,Ind%:Professional & Business Services,Ind%:Public Administration,Ind%:Service Producing,Ind%:Total,"Ind%:Trade, Transportation, & Utilities",GEO_ID,Source
0,"Adams city, Tennessee",2019-01-01,2019Q1,1.305952,16.306345,3.022547,2.244575,0.035368,9.184870,0.000106,0.938517,2.794552,1.051732,3.650866,36.046279,60.957684,22.666830,2.142391,26.750269,4.958434,3.682185,0.058021,15.067617,0.000174,1.539620,4.584413,1.725348,5.989180,59.133282,NaN,37.184533,1600000US4700200,JobsEQ
1,"Adams city, Tennessee",2019-04-01,2019Q2,0.973770,16.215050,3.007493,1.897441,0.032259,8.781033,0.000102,0.923569,2.843634,1.138403,3.264177,35.282049,59.517389,22.337899,1.636111,27.244222,5.053134,3.188045,0.054201,14.753727,0.000171,1.551764,4.777821,1.912723,5.484410,59.280237,NaN,37.531718,1600000US4700200,JobsEQ
2,"Adams city, Tennessee",2019-07-01,2019Q3,0.645157,16.136415,2.970006,1.567421,0.032931,8.465746,0.000098,0.922167,2.914760,1.237681,2.874473,34.632012,57.964825,21.765392,1.113014,27.838289,5.123807,2.704090,0.056811,14.604972,0.000169,1.590907,5.028498,2.135227,4.958995,59.746599,NaN,37.549310,1600000US4700200,JobsEQ
3,"Adams city, Tennessee",2019-10-01,2019Q4,0.315193,16.983101,2.901212,1.225575,0.033157,8.138603,0.000094,0.910288,2.883152,1.335975,2.484836,34.760037,56.900578,20.914967,0.553936,29.846975,5.098738,2.153888,0.058272,14.303199,0.000165,1.599787,5.067000,2.347911,4.366979,61.089075,NaN,36.757037,1600000US4700200,JobsEQ
4,"Adams city, Tennessee",2020-01-01,2020Q1,0.315552,18.009454,2.928454,1.193624,0.036952,8.267085,0.000094,0.877978,3.012317,1.356741,2.464372,36.075376,57.990243,20.721243,0.544147,31.056008,5.049909,2.058319,0.063721,14.255993,0.000162,1.514010,5.194523,2.339602,4.249633,62.209389,NaN,35.732292,1600000US4700200,JobsEQ


In [116]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 45 columns):
 #   Column                                                              Non-Null Count  Dtype         
---  ------                                                              --------------  -----         
 0   NAME                                                                1462 non-null   object        
 1   TimeStamp                                                           1462 non-null   datetime64[ns]
 2   Time                                                                1462 non-null   object        
 3   Ind:Construction                                                    1462 non-null   float64       
 4   Ind:Education & Health Services                                     1462 non-null   float64       
 5   Ind:Financial Activities                                            1462 non-null   float64       
 6   Ind:Goods Producing                                     

In [117]:
#export to the SQLite database as only the JobsEQ quarterly data
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_IndustryEmployment_Quarterly', conn, if_exists = 'replace', index = False)

1462